In [ ]:
import pl.mareklangiewicz.kommand.*
import pl.mareklangiewicz.kommand.coreutils.Ls
import pl.mareklangiewicz.kommand.coreutils.LsOpt.*
import pl.mareklangiewicz.kommand.coreutils.ls
import pl.mareklangiewicz.kommand.gnome.*
import pl.mareklangiewicz.kommand.gnome.GnomeExt.Cmd.list

println("start")

CliPlatform.SYS.start(gnometerm(bash(gnomeext(list), pause = true)))
CliPlatform.SYS.start(gnometerm(bash(ls { -LongFormat; -All }, pause = true)))

println("end")

val x = "x"


In [ ]:
import pl.mareklangiewicz.kommand.CliPlatform.Companion.SYS
import pl.mareklangiewicz.kommand.konfig.*

SYS.run {
    konfigInUserHomeConfigDir().printAll()
}

In [ ]:
import pl.mareklangiewicz.kommand.demos.*
import pl.mareklangiewicz.kommand.*
import pl.mareklangiewicz.kommand.samples.Samples
import pl.mareklangiewicz.kommand.coreutils.*

//CliPlatform.SYS.LangaraREPL()
CliPlatform.SYS.run { 
//    Samples.CoreUtils.Ls.lsWithHidden.exec().printlns()
    ls("..", withHidden = true, style = Ls.Option.indicatorStyle.SLASH).exec().printlns()
 }

In [ ]:
import pl.mareklangiewicz.kommand.CliPlatform.Companion.SYS
import pl.mareklangiewicz.kommand.bash
import pl.mareklangiewicz.kommand.gnome.startInGnomeTermIfUserConfirms
import pl.mareklangiewicz.kommand.printlns
import pl.mareklangiewicz.kommand.samples.Samples

SYS.run {
//    val kmd = Samples.GitHub.help
//    val kmd = Samples.GitHub.secretSetFakeSecretInAbcdK
    val kmd = Samples.GitHub.secretListForAbcdK
    startInGnomeTermIfUserConfirms(kmd)
}
